In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from numpy import random
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import time
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

sns.set_palette('Dark2')
plt.rcParams['figure.figsize'] = (8,6)
%matplotlib inline


In [2]:
wines = pd.read_csv('../WineData/cleanWineData.csv',index_col=0)
wines.head()

,Ageing,Alcohol,BlendPercent,CommonName,FSO2,Latitude,Longitude,MainVariety,Origin,RS,...,Vintage,Vintage_dateTime,WineBody,WineMaker,WineName,WineStyle,WineTaste,WineType,Wooded,pH
0,1-3 years,13.50,Pinot Gris,NaN,NaN,NaN,NaN,Pinot Gris,Robertson,1.8,...,2002.0,2002-01-01 00:00:00,NaN,NaN,Van Loveren Pinot Gris 2002,NaN,NaN,White,NaN,NaN
1,1-5 years,13.50,Chardonnay,NaN,NaN,NaN,NaN,Chardonnay,Robertson,1.7,...,2002.0,2002-01-01 00:00:00,NaN,NaN,Van Loveren Limited Release Chardonnay 2002,NaN,NaN,White,NaN,NaN
2,NaN,NaN,Blend,NaN,NaN,NaN,NaN,NaN,Walker Bay,NaN,...,NaN,NaN,NaN,NaN,Whalehaven Baleine Noir NV,NaN,NaN,NaN,NaN,NaN
3,This wine will improve in complexity over the ...,15.00,100% Shiraz,NaN,NaN,NaN,NaN,Shiraz,Coastal,2.5,...,2001.0,2001-01-01 00:00:00,NaN,Andries Blake,Indalo Shiraz 2001,NaN,NaN,Red,NaN,NaN
4,NaN,13.25,Pinotage,NaN,NaN,NaN,NaN,Pinotage,Coastal,1.7,...,NaN,NaN,NaN,Andries Blake,Indalo Pinotage,NaN,NaN,Red,NaN,NaN


# Extracting flavours

First I need to obtain a list of words in the tasting notes

In [37]:
from nltk.corpus import stopwords
import nltk
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tokenize import word_tokenize
import re

def review_to_wordlist( review, remove_stopwords=False ):  #from https://www.kaggle.com/c/word2vec-nlp-tutorial
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.

    review_text = review
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    review_text = re.sub('gold', '', review_text)
    review_text = re.sub('silver', '', review_text)
    review_text = re.sub('bronze', '', review_text)
    review_text = re.sub('[0-9]+', '', review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)


# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False ):   #from https://www.kaggle.com/c/word2vec-nlp-tutorial
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    if isinstance(review,float):
        return ['']
    else:
        review = review.decode('utf-8')
        raw_sentences = tokenizer.tokenize(review.strip())

    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence,remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

def sentences_to_list(sentences):
    newList = []
    skipped = 0
    for s in range(len(sentences)):
        try:
            newList += sentences[s]
        except:
            skipped +=1
    print 'Skipping ',skipped,' lines since they are NoneType'
    return newList

def unicodeStrToStr(x):
    try:
        x = map(str, x)
    except:
        pass
    return x

There are also some words in the tasting notes that are superfluous or unhelpful, so I have defined some functions to remove these

In [ ]:
with open('wine_stopwords.txt') as f:
    wine_stopwords = []
    lines = f.readlines()
    for line in lines:
        wine_stopwords.append(str(line.split('\n')[0]))

def removeWineStopWords(descriptionList):
    global wine_stopwords
    if isinstance(descriptionList,float):
        return descriptionList
    else:
        list_u = []
        [list_u.append(v) for v in descriptionList if v not in wine_stopwords]
        if 'tint' in descriptionList:
            c_index = list_u.index("tint")
            list_u = [list_u[:c_index+1],list_u[c_index+1:]]
        elif 'colour' in descriptionList:
            c_index = list_u.index("colour")
            list_u = [list_u[:c_index+1],list_u[c_index+1:]] 
        elif 'color' in descriptionList:
            c_index = list_u.index("color")
            list_u = [list_u[:c_index+1],list_u[c_index+1:]] 
        return list_u
    
def getFlavoursnotColors(description):
    if isinstance(description,float):
        return []
    elif len(description)==0:
        return description        
    elif isinstance(description[0],unicode):
        return description
    else:
        for i in description:
            if 'tint' in i:
                pass
            elif 'colour' in i:
                pass
            elif 'color' in i:
                pass
            else:
                return i

Now the normal stopwords are removed...

In [19]:
descriptions = wines.TastingNotes.str.lower()
descriptionWords = descriptions.apply(lambda x: review_to_sentences(x,tokenizer,True))
descriptionWords = descriptionWords.apply(sentences_to_list)
descriptionWords.head()

0    [pinot, gris, full, bodied, well, balanced, wi...
1    [full, bodied, well, balanced, nutty, wine, ag...
2    [special, release, lightly, wooded, blend, cab...
3    [intense, dark, ruby, red, colour, spicy, sadd...
4    [michelangelo, awards, medalvery, dark, ruby, ...
Name: TastingNotes, dtype: object

Followed by the wine stopwords. I also make sure that I am not using visible descriptions in my analysis

In [20]:
cleanDescriptions = descriptionWords.apply(removeWineStopWords)

cleanDescriptionFlavours = cleanDescriptions.apply(getFlavoursnotColors)
cleanDescriptionFlavours.head()

/Users/lauren/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:85: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


0                                                   []
1                                [nutty, oak, buttery]
2    [wooded, herby, mintiness, cherries, cassis, f...
3    [spicy, saddle, bag, complimented, oak, berry,...
4                    [banana, red, oak, berry, wooden]
Name: TastingNotes, dtype: object

I then read in the flavour profiles

In [34]:
flavourProfiles = pd.read_csv('flavourProfiles.csv',skiprows=1,names=['Profile','Varieties'])
flavourProfiles.Varieties = flavourProfiles.Varieties.str.lower()#.str.split(',')
# profileText = flavourProfiles.Varieties.str.replace(',',' ').str.cat(sep='')
flavourProfiles


,Profile,Varieties
0,Flower,"iris,peony,elderflower,acacia,lilac,jasmine,ho..."
1,Citrus,"lime,lemon,grapefruit,orange,marmalade,lemons,..."
2,Tree Fruit,"quince,apple,pear,nectarine,peach,apricot,pers..."
3,Tropical Fruit,"pineapple,mango,guava,passion fruit,lychee,bub..."
4,Red Fruit,"cranberry,red plum,pomegranate,sour cherry,str..."
5,Black Fruit,"boysenberry,blackcurrant,black cherry,plum,bla..."
6,Dried Fruit,"raisin,fig,date,fruit cake,figs,fruitcake"
7,Noble Rot,"bees wax,ginger,honey"
8,Spice,"white pepper,red pepper,black pepper,cinnamon,..."
9,Vegetable,"grass,tomato leaf,gooseberry,bell pepper,jalap..."


In [35]:
from difflib import SequenceMatcher

def similarList(lst, b):
    similarities = []
    for i in range(len(lst)):
        similarities.append(SequenceMatcher(None, lst, b).ratio())
    return similarities

def assignFlavourProfile(flavourList,flavourProfiles,winestopwords):
    """
    This function looks at the leftover words in the tasting notes and either:
    1) Finds extra stopwords that were missed
    2) Updates the flavour profile to recognise new (possibly misspelled) descriptors
    3) Adds the word to a dictionary so tasting notes can be mapped to Flavour Profiles
    """
    profileDict= {}
    #address easy fits and multi-word flavours first
    for n in nltk.bigrams(flavourList):
        combinedBigram = n[0]+' '+n[1]
        if flavourProfiles['Varieties'].str.contains(combinedBigram).any():
            location =  flavourProfiles['Profile'].where(flavourProfiles['Varieties'].str.contains(combinedBigram))
            profileDict[combinedBigram] = list(location.dropna())[0]    
            
    #now we consider the flavours with frivolous adjectives
    for n in range(len(flavourList)):
        if n>0 and n%50 ==0:
            contin = raw_input('Stop or continue?[[s]/c]')
        else: 
            contin = 'c'
        if contin == 'c':                             
            if flavourList[n] in profileDict:
                pass
            else:
                f = flavourList[n]
                profile = []
                if flavourProfiles['Varieties'].str.contains(f).any():

                    check = flavourProfiles['Varieties'].where(flavourProfiles['Varieties'].str.contains(f)).dropna()
                    checkList = list(check)[0]
                    location =  flavourProfiles['Profile'].where(flavourProfiles['Varieties'].str.contains(f))
                    profile.append(list(location.dropna())[0]) 
                else:
                    print f, 'does not seem to belong to any profile.'
                    print 'Profiles are\n',flavourProfiles.Profile
                    user_input = raw_input('What profile should %s belong to? Return "w" if this is an winestopword, "p" if you want to see the previous word or "n" if you want to see the next word, else return the profile number'%f)
                    if user_input == 'w': 
                        winestopwords.append(f)
                    elif user_input == 'p': 
                        try:
                            f = flavourList[n-1]+' '+flavourList[n]
                        except:
                            pass
                        user_input = raw_input('What profile should %s belong to in context %s? Return x if still cannot be classified, else the profile number'%(f,flavourList))
                    elif user_input == 'n': 
                        try:
                            f = flavourList[n]+' '+flavourList[n+1]
                        except:
                            pass
                        user_input = raw_input('What profile should %s belong to? Return x if still cannot be classified, else the profile number'%f)
                    if user_input not in ['w','x']:
                        user_input = int(user_input)
                        flavourProfiles.Varieties[user_input]+=','+str(f)
                        profile.append(flavourProfiles.Profile[user_input])
                if len(profile)>1:
                    print 'Flavour is ',flavourList[n],'and chosen profiles are',profile
                    profileChoice = raw_input('What profile should  be chosen? Please choose index or "n" to go back to original list')
                    if profileChoice == 'n':
                        print 'Profiles are\n',flavourProfiles.Profile
                        profileChoice = raw_input('What profile should %s belong to? Return the profile number'%flavourList[n])
                        profile = list(flavourProfiles.Profile[int(profileChoice)])
                    else: 
                        profile = profile[int(profileChoice)]
                    profile = list(profileChoice)
                if len(profile) > 0:
                    profileDict[flavourList[n]] = profile[0]
        else:
            return profileDict,winestopwords,flavourList
    return profileDict,winestopwords

Now I combine all words in the descriptions and take the set of descriptors to check for stoppwords and create the dictionary to map tasting notes to profiles

In [39]:
uniqueFlavours = sentences_to_list(list(cleanDescriptionFlavours))
print 'Description words ',len(uniqueFlavours)
uniqueFlavours = list(set(uniqueFlavours))
print 'Unique description words: ',len(uniqueFlavours)

Skipping  106  lines since they are NoneType
Description words  80843
Unique description words:  6120


In [23]:
assignDict,wineStopWords = assignFlavourProfile(uniqueFlavours,flavourProfiles,wine_stopwords)
print assignDict

with open('wine_stopwords.txt','w') as f:
    for a in wineStopWords:
        if a == wineStopWords[-1]:
            f.write(a)
        else:
            f.write(a+'\n')
flavourProfiles.to_csv('flavourProfiles.csv',columns =['Profile','Varieties'])

Stop or continue?[[s]/c]s
{'melons': 'Tropical Fruit', 'fru': 'Citrus', 'nuttyness': 'Oak Aging', 'saltiness': 'Brettanomyces', 'melony': 'Tropical Fruit', 'mocha': 'General Aging', 'cocoa': 'General Aging', 'earthiness': 'Earth', 'boiled': 'Sulfides & Mercaptans', 'carbon': 'Earth', 'woody': 'Oak Aging', 'corns': 'Spice', 'chili': 'Spice', 'caper': 'Vegetable', 'rd': 'Tree Fruit', 'topical': 'Tropical Fruit', 'frui': 'Citrus', 'fig': 'Dried Fruit', 'appricots': 'Tree Fruit', 'eucalypt': 'Spice', 'har': 'Sulfides & Mercaptans', 'rice': 'Spice', 'bitter': 'Vegetable', 'ri': 'Flower', 'elderflower': 'Flower', 'buchu': 'Vegetable', 'grapef': 'Citrus', 'blossoms': 'Flower', 'khakibos': 'Spice', 'herb': 'Vegetable', 'slate': 'Earth', 'jasmine': 'Flower', 'shale': 'Earth', 'spic': 'Spice', 'figs': 'Dried Fruit', 'oleic': 'Earth', 'berrylike': 'Red Fruit', 'stonefruits': 'Tree Fruit', 'sulphur': 'Sulfides & Mercaptans', 'grapey': 'Tree Fruit', 'wooded': 'Oak Aging', 'chocochino': 'General Agi

In [25]:
wines['clean_TastingNotes'] = cleanDescriptionFlavours 
wines.clean_TastingNotes.head()

0                                                   []
1                                [nutty, oak, buttery]
2    [wooded, herby, mintiness, cherries, cassis, f...
3    [spicy, saddle, bag, complimented, oak, berry,...
4                    [banana, red, oak, berry, wooden]
Name: clean_TastingNotes, dtype: object

### Map tasting notes to profile

In [108]:
def flavour_to_profile(assignDict,row):
    """
    Take the flavours identified from the tasting notes and map them to profiles
    The result will be a table where the row will show '1' if the profile is present in that wine and '0' otherwise
    """
    profiles = []
    for f in row.clean_TastingNotes:
        try:
            if row[assignDict[f]] ==1:
                pass
            else:
                row[assignDict[f]] +=1
        except:
            pass
    return row
    
wine_FlavourProfiles = wines[['WineName','WineType','Vintage_dateTime','clean_TastingNotes']].dropna()
wine_FlavourProfiles.clean_TastingNotes = wine_FlavourProfiles.clean_TastingNotes.apply(lambda y: np.nan if len(y)==0 else y)
for p in flavourProfiles.Profile:
    wine_FlavourProfiles[p] = 0
wine_FlavourProfiles = wine_FlavourProfiles.dropna()
wine_FlavourProfiles = wine_FlavourProfiles.apply(lambda row: flavour_to_profile(assignDict,row), axis = 1)
wine_FlavourProfiles.describe()

,Flower,Citrus,Tree Fruit,Tropical Fruit,Red Fruit,Black Fruit,Dried Fruit,Noble Rot,Spice,Vegetable,Earth,Microbial,Oak Aging,General Aging,Cork Taint(TCA),Sulfides & Mercaptans,Brettanomyces,Madeirized (orCooked),Volatile Acidity (AceticAcid)
count,6438.000000,6438.000000,6438.000000,6438.000000,6438.000000,6438.0,6438.000000,6438.0,6438.000000,6438.000000,6438.000000,6438.0,6438.000000,6438.000000,6438.0,6438.000000,6438.000000,6438.0,6438.000000
mean,0.010873,0.002019,0.002175,0.003883,0.000311,0.0,0.040851,0.0,0.004815,0.011960,0.004039,0.0,0.021435,0.025784,0.0,0.001243,0.000466,0.0,0.000155
std,0.103713,0.044894,0.046585,0.062199,0.017624,0.0,0.197961,0.0,0.069229,0.108715,0.063426,0.0,0.144841,0.158504,0.0,0.035232,0.021583,0.0,0.012463
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,0.0,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.0,1.000000,1.000000,0.0,1.000000


In [109]:
wines.info()
wines.to_csv('../WineData/WineData_withFlavours.csv', columns = wines.columns, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7745 entries, 0 to 8776
Data columns (total 25 columns):
Ageing                1592 non-null object
Alcohol               4080 non-null float64
BlendPercent          4541 non-null object
CommonName            3085 non-null object
FSO2                  402 non-null float64
Latitude              3046 non-null object
Longitude             3046 non-null object
MainVariety           7634 non-null object
Origin                6809 non-null object
RS                    3892 non-null float64
TA                    3858 non-null float64
TSO2                  452 non-null float64
TastingNotes          6864 non-null object
VA                    404 non-null object
Vintage               7392 non-null float64
Vintage_dateTime      7382 non-null object
WineBody              1716 non-null object
WineMaker             3949 non-null object
WineName              7745 non-null object
WineStyle             2493 non-null object
WineTaste             1697 non

In [112]:
print 'Number of instances of each profile: '
print wine_FlavourProfiles.sum(axis = 0)
wine_sums = wine_FlavourProfiles.sum(axis = 1)
print 'Number of wines where a profile has been identified: ',sum(wine_sums > 0)
wine_FlavourProfiles.to_csv('../WineData/wine_FlavourPofiles.csv', columns = wine_FlavourProfiles.columns, encoding='utf-8')

Number of instances of each profile: 
WineName                         Van Loveren Limited Release Chardonnay 2002Ind...
WineType                         WhiteRedWhiteRedRedWhiteWhiteRedRedRedWhiteRed...
Vintage_dateTime                 2002-01-01 00:00:002001-01-01 00:00:002004-01-...
clean_TastingNotes               [nutty, oak, buttery, spicy, saddle, bag, comp...
Flower                                                                          70
Citrus                                                                          13
Tree Fruit                                                                      14
Tropical Fruit                                                                  25
Red Fruit                                                                        2
Black Fruit                                                                      0
Dried Fruit                                                                    263
Noble Rot                                        